<br>

# Milestone1_3. Data Preparation & Feature Engineering

Load, Clean Data Stored from the Previous Step and Create More Features

## Team #30

In [1]:
# nltk.download('vader_lexicon')

In [2]:
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
from nltk.sentiment import SentimentIntensityAnalyzer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


<br>

## 3.1. Restaurant Table - Cleaning & Feature Engineering

In [3]:
# Load and check data shape, data type
restaurant = pd.read_csv('data/restaurant.csv', low_memory=False)
print(restaurant.shape)
print(restaurant.dtypes)

(49875, 51)
business_id                    object
name                           object
address                        object
city                           object
state                          object
postal_code                   float64
latitude                      float64
longitude                     float64
stars                         float64
review_count                    int64
is_open                         int64
categories                     object
RestaurantsDelivery            object
OutdoorSeating                 object
BusinessAcceptsCreditCards     object
BikeParking                    object
RestaurantsPriceRange2         object
RestaurantsTakeOut             object
ByAppointmentOnly              object
WiFi                           object
Alcohol                        object
Caters                         object
RestaurantsAttire              object
RestaurantsReservations        object
GoodForKids                    object
CoatCheck                      object


### 3.1.1. Drop columns that are irrelevant

In [4]:
# Drop restaurant address as it is not relevant to the analysis
restaurant.drop(columns=['address'], inplace=True)

In [5]:
# Drop columns for schedule as the data is quite messy and not very relevant to the analysis
restaurant.drop(columns=['Monday', 'Tuesday',
       'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)

### 3.1.2. Handling missing values

#### 3.1.2.1. For essential columns, drop records with missing values

In [6]:
# Drop rows missing zip code info, which is important for the next steps
restaurant.dropna(subset=['postal_code'], inplace=True)

#### 3.1.2.2. Check & drop non-essential columns with >50% missingness

In [7]:
s = (restaurant.isna().sum() / restaurant.shape[0]).round(3)
s[s>0.5]

ByAppointmentOnly            0.934
CoatCheck                    0.917
DogsAllowed                  0.775
RestaurantsTableService      0.620
WheelchairAccessible         0.737
HappyHour                    0.746
DriveThru                    0.878
BusinessAcceptsBitcoin       0.865
Smoking                      0.933
GoodForDancing               0.930
BYOB                         0.917
Corkage                      0.933
BYOBCorkage                  0.971
RestaurantsCounterService    1.000
Open24Hours                  1.000
AgesAllowed                  0.998
AcceptsInsurance             1.000
dtype: float64

In [8]:
restaurant.ByAppointmentOnly.value_counts()

False    3174
True      121
None        1
Name: ByAppointmentOnly, dtype: int64

In [9]:
restaurant.DogsAllowed.value_counts()

False    8422
True     2758
None       13
Name: DogsAllowed, dtype: int64

In [10]:
restaurant.HappyHour.value_counts()

True     8296
False    4379
None        1
Name: HappyHour, dtype: int64

In [11]:
restaurant.DriveThru.value_counts()

True     3387
False    2307
None      367
Name: DriveThru, dtype: int64

In [12]:
# Drop columns with more than 50% na values
restaurant.drop(columns=s[s>0.5].index, inplace=True)

#### 3.1.2.3. One-Hot Encoding & Replace remaining missing values based on judgement calls

In [13]:
print(f'{restaurant.RestaurantsReservations.isna().sum()}')
restaurant.RestaurantsDelivery.value_counts()

8716


True     26391
False    16577
None      2589
Name: RestaurantsDelivery, dtype: int64

In [14]:
# Assume False for na and None and covert to binary
restaurant['RestaurantsDelivery'] = np.where(restaurant.RestaurantsDelivery=='True', 1, 0)

In [15]:
print(f'{restaurant.OutdoorSeating.isna().sum()}')
restaurant.OutdoorSeating.value_counts()

8098


False    20703
True     19357
None      1698
Name: OutdoorSeating, dtype: int64

In [16]:
# Assume False for na and None and covert to binary
restaurant.OutdoorSeating = np.where(restaurant.OutdoorSeating=='True', 1, 0)

In [17]:
print(f'{restaurant.BusinessAcceptsCreditCards.isna().sum()}')
restaurant.BusinessAcceptsCreditCards.value_counts()

4426


True     44003
False     1404
None        23
Name: BusinessAcceptsCreditCards, dtype: int64

In [18]:
# Assume False for na and None and covert to binary
restaurant.BusinessAcceptsCreditCards = np.where(restaurant.BusinessAcceptsCreditCards=='True', 1, 0)

In [19]:
print(f'{restaurant.BikeParking.isna().sum()}')
restaurant.BikeParking.value_counts()

15891


True     25024
False     8918
None        23
Name: BikeParking, dtype: int64

In [20]:
# Assume False for na and None and covert to binary
restaurant.BikeParking = np.where(restaurant.BikeParking=='True', 1, 0)

In [21]:
print(f'{restaurant.RestaurantsPriceRange2.isna().sum()}')
restaurant.RestaurantsPriceRange2.value_counts()

7274


2       21739
1       19139
3        1512
4         179
None       13
Name: RestaurantsPriceRange2, dtype: int64

In [22]:
# Change na or None value to '99'; keep as a categorical variable
restaurant['RestaurantsPriceRange'] = np.where(restaurant.RestaurantsPriceRange2=='None', '99', 
                                           np.where(restaurant.RestaurantsPriceRange2.isna(), '99', 
                                                   restaurant.RestaurantsPriceRange2))

In [23]:
restaurant.RestaurantsPriceRange.value_counts()

2     21739
1     19139
99     7287
3      1512
4       179
Name: RestaurantsPriceRange, dtype: int64

In [24]:
restaurant.groupby('RestaurantsPriceRange').stars.mean()

RestaurantsPriceRange
1     3.302027
2     3.591012
3     3.693452
4     3.689944
99    3.813298
Name: stars, dtype: float64

In [25]:
restaurant['expensive'] = np.where(restaurant.RestaurantsPriceRange.isin(['3','4']), 1, 0)

In [26]:
restaurant.expensive.value_counts()

0    48165
1     1691
Name: expensive, dtype: int64

In [27]:
restaurant.groupby('expensive').stars.mean()

expensive
0    3.509810
1    3.693081
Name: stars, dtype: float64

In [28]:
restaurant.drop(columns=['RestaurantsPriceRange2'],inplace=True)

In [29]:
print(f'{restaurant.RestaurantsTakeOut.isna().sum()}')
restaurant.RestaurantsTakeOut.value_counts()

3295


True     43378
False     2104
None      1079
Name: RestaurantsTakeOut, dtype: int64

In [30]:
# Assume False for na and None and covert to binary
restaurant.RestaurantsTakeOut = np.where(restaurant.RestaurantsTakeOut=='True', 1, 0)

In [31]:
print(f'{restaurant.WiFi.isna().sum()}')
restaurant.WiFi.value_counts()

13741


u'free'    14255
u'no'      10926
'no'        5451
'free'      5207
u'paid'      152
'paid'        92
None          32
Name: WiFi, dtype: int64

In [32]:
# Convert to binary based on if free WiFi is stated available
restaurant['free_WiFi'] = np.where(restaurant.WiFi.isna(), 0,
                                   np.where(restaurant.WiFi.str.contains('free'), 1, 0))

In [33]:
restaurant.drop(columns=['WiFi'], inplace=True)

In [34]:
print(f'{restaurant.Alcohol.isna().sum()}')
restaurant.Alcohol.value_counts()

11543


u'none'             15162
u'full_bar'         10747
'none'               4540
u'beer_and_wine'     4391
'full_bar'           2367
'beer_and_wine'      1077
None                   29
Name: Alcohol, dtype: int64

In [35]:
# Convert to binary based on if alcohol is stated available
restaurant.Alcohol = np.where(restaurant.Alcohol.isna(), 0,
                                   np.where(restaurant.Alcohol.str.contains('full_bar'), 1, 
                                            np.where(restaurant.Alcohol.str.contains('beer_and_wine'), 1,0)))

In [36]:
print(f'{restaurant.Caters.isna().sum()}')
restaurant.Caters.value_counts()

16652


True     19917
False    13243
None        44
Name: Caters, dtype: int64

In [37]:
# Assume False for na and None and covert to binary
restaurant.Caters = np.where(restaurant.Caters=='True', 1, 0)

In [38]:
print(f'missing: {restaurant.RestaurantsAttire.isna().sum()}')
restaurant.RestaurantsAttire.value_counts()

missing: 12645


u'casual'    21849
'casual'     14538
u'dressy'      469
'dressy'       257
'formal'        37
None            33
u'formal'       28
Name: RestaurantsAttire, dtype: int64

In [39]:
restaurant.groupby('RestaurantsAttire').stars.mean()

RestaurantsAttire
'casual'     3.422961
'dressy'     3.704280
'formal'     2.918919
None         3.151515
u'casual'    3.468992
u'dressy'    3.731343
u'formal'    3.142857
Name: stars, dtype: float64

In [40]:
# Covert to binary with 1 for dressy and 0 for the rest
restaurant['attire_dressy'] = np.where(restaurant.RestaurantsAttire.isna(), 0,
                                       np.where(restaurant.RestaurantsAttire.str.contains('dressy'),1,0))

restaurant['attire_dressy'].value_counts()

0    49130
1      726
Name: attire_dressy, dtype: int64

In [41]:
restaurant.drop(columns=['RestaurantsAttire'], inplace=True)

In [42]:
print(f'{restaurant.RestaurantsReservations.isna().sum()}')
restaurant.RestaurantsReservations.value_counts()

8716


False    27528
True     13372
None       240
Name: RestaurantsReservations, dtype: int64

In [43]:
# Assume False for na and None and covert to binary
restaurant.RestaurantsReservations = np.where(restaurant.RestaurantsReservations=='True',1,0)

In [44]:
print(f'{restaurant.GoodForKids.isna().sum()}')
restaurant.GoodForKids.value_counts()

10775


True     33216
False     5846
None        19
Name: GoodForKids, dtype: int64

In [45]:
# Assume False for na and None and covert to binary
restaurant.GoodForKids = np.where(restaurant.GoodForKids=='True',1,0)

In [46]:
print(f'{restaurant.RestaurantsGoodForGroups.isna().sum()}')
restaurant.RestaurantsGoodForGroups.value_counts()

10314


True     33887
False     5640
None        15
Name: RestaurantsGoodForGroups, dtype: int64

In [47]:
# Assume False for na and None and covert to binary
restaurant.RestaurantsGoodForGroups = np.where(restaurant.RestaurantsGoodForGroups=='True',1,0)

In [48]:
print(f'{restaurant.HasTV.isna().sum()}')
restaurant.HasTV.value_counts()

9849


True     30295
False     9695
None        17
Name: HasTV, dtype: int64

In [49]:
# Assume False for na and None and covert to binary
restaurant.HasTV = np.where(restaurant.HasTV=='True',1,0)

In [50]:
print(f'{restaurant.NoiseLevel.isna().sum()}')
restaurant.NoiseLevel.value_counts()

16574


u'average'      19217
u'quiet'         5670
'average'        4235
u'loud'          1867
'quiet'          1094
u'very_loud'      690
'loud'            363
'very_loud'       113
None               33
Name: NoiseLevel, dtype: int64

In [51]:
# Convert missing value to 'unknown'
restaurant.NoiseLevel = np.where(restaurant.NoiseLevel.isna() | (restaurant.NoiseLevel=='None'), 'unknown',
                                np.where(restaurant.NoiseLevel.str.contains('quiet'), 'quiet',
                                        np.where(restaurant.NoiseLevel.str.contains('average'), 'average','loud')))

restaurant.NoiseLevel.value_counts()

average    23452
unknown    16607
quiet       6764
loud        3033
Name: NoiseLevel, dtype: int64

In [52]:
restaurant.groupby('NoiseLevel').stars.mean()

NoiseLevel
average    3.554025
loud       3.138147
quiet      3.592992
unknown    3.500030
Name: stars, dtype: float64

In [53]:
restaurant['noise_loud'] = np.where(restaurant.NoiseLevel=='loud', 1, 0)

restaurant.noise_loud.value_counts()

0    46823
1     3033
Name: noise_loud, dtype: int64

In [54]:
print(restaurant.shape)
print(f'missing values: {restaurant.isna().sum().sum()}')

(49856, 28)
missing values: 0


<br>

## 3.2. Join Table Demographics & Create New Features

In [55]:
# Load and check data shape, data type
demographics = pd.read_csv('data/demographics.csv', low_memory=False)
print(demographics.shape)
print(demographics.dtypes)

(803, 10)
median_household_income         int64
population                      int64
household_cnt                   int64
education_total                 int64
bachelors_degree                int64
median_age                    float64
population_hispanic_latino      int64
population_white                int64
population_asian                int64
zip_code                        int64
dtype: object


In [56]:
demographics.isna().sum()

median_household_income       0
population                    0
household_cnt                 0
education_total               0
bachelors_degree              0
median_age                    0
population_hispanic_latino    0
population_white              0
population_asian              0
zip_code                      0
dtype: int64

### 3.2.1. Check errors & outliers

In [57]:
demographics.describe()

,median_household_income,population,household_cnt,education_total,bachelors_degree,median_age,population_hispanic_latino,population_white,population_asian,zip_code
count,8.030000e+02,803.00000,803.000000,803.000000,803.000000,8.030000e+02,803.000000,803.000000,803.000000,803.000000
mean,-2.482294e+07,22442.67995,8900.798257,15586.884184,3622.198007,-9.132380e+06,3080.188045,14691.877958,1030.471980,38983.298879
std,1.265234e+08,15956.03988,6241.768935,10840.566256,2972.701775,7.753945e+07,4983.997146,11392.918007,1495.792344,24856.106015
min,-6.666667e+08,0.00000,0.000000,0.000000,0.000000,-6.666667e+08,0.000000,0.000000,0.000000,7836.000000
25%,5.891150e+04,9690.50000,3876.500000,6929.500000,1330.500000,3.645000e+01,414.500000,5688.500000,161.000000,19086.500000
50%,8.051900e+04,19863.00000,7957.000000,13967.000000,3038.000000,4.000000e+01,1292.000000,13006.000000,519.000000,33711.000000
75%,1.034425e+05,32344.00000,12898.000000,22168.500000,5121.000000,4.420000e+01,3375.000000,21210.500000,1294.500000,62248.500000
max,2.500010e+05,99800.00000,36255.000000,63213.000000,18106.000000,8.220000e+01,48805.000000,61814.000000,11682.000000,96161.000000


In [58]:
# Drop entries with zero population or negative median_household_income

demographics = demographics[(demographics.population>0) 
                            & (demographics.median_household_income>0)
                            & (demographics.bachelors_degree>0)]

In [59]:
demographics.population.quantile([0.05,0.1,0.2,0.5,0.8,0.9,0.95])

0.05     2400.85
0.10     5159.70
0.20     9078.60
0.50    20395.50
0.80    35551.80
0.90    44811.20
0.95    53208.35
Name: population, dtype: float64

### 3.2.2. Normalize data

In [60]:
# Create new feature to normalize data

demographics['bachelors_pcnt'] = round(demographics['bachelors_degree'] / demographics['education_total'],4)
demographics['education_pcnt'] = round(demographics['education_total'] / demographics['population'],4)

In [61]:
demographics.drop(columns=['bachelors_degree','education_total'], inplace=True)

In [62]:
demographics.describe()

,median_household_income,population,household_cnt,median_age,population_hispanic_latino,population_white,population_asian,zip_code,bachelors_pcnt,education_pcnt
count,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000
mean,86699.330311,23305.682642,9255.911917,40.815674,3198.781088,15257.651554,1067.908031,38777.182642,0.232036,0.707181
std,36459.743499,15664.225570,6103.676566,6.949441,5046.893928,11253.757353,1512.124226,24691.447089,0.093243,0.066577
min,16916.000000,67.000000,33.000000,20.600000,0.000000,67.000000,0.000000,7836.000000,0.018800,0.358000
25%,61262.500000,10989.250000,4313.750000,36.700000,490.750000,6607.750000,189.000000,19080.500000,0.160200,0.671775
50%,82546.000000,20395.500000,8287.500000,40.050000,1359.000000,13548.000000,566.000000,33711.500000,0.229850,0.705650
75%,104888.000000,32682.250000,13068.250000,44.200000,3515.000000,21681.000000,1354.000000,62236.750000,0.297225,0.739500
max,250001.000000,99800.000000,36255.000000,82.200000,48805.000000,61814.000000,11682.000000,96161.000000,0.608800,1.000000


### 3.2.3. Merge data

In [63]:
# Inner join restaurant with demographics tables

master1 = restaurant.rename(columns={'postal_code':'zip_code'}).merge(demographics, how='inner', on='zip_code')

In [64]:
print(master1.shape)
print(demographics.shape)
print(master1.business_id.nunique())
print(master1.zip_code.nunique())

(49369, 37)
(772, 10)
49369
772


In [65]:
restaurant_count_byZipCode = (master1.groupby('zip_code').business_id.count().reset_index()
                              .rename(columns={'business_id':'restaurant_count'}))

In [66]:
master1 = master1.merge(restaurant_count_byZipCode, how='left', on='zip_code')
master1.shape

(49369, 38)

### 3.2.4. Create new features

In [67]:
master1['population_perRestaurant'] = round(master1['population'] / master1['restaurant_count'])
master1['household_perRestaurant'] = round(master1['household_cnt'] / master1['restaurant_count'])

In [68]:
master1['hispanic_latino_pcnt'] = round(master1['population_hispanic_latino'] / master1['population'],3)
master1['white_pcnt'] = round(master1['population_white'] / master1['population'],3)
master1['asian_pcnt'] = round(master1['population_asian'] / master1['population'],3)

In [69]:
# Store temp master file

master1.to_csv('data/master1.csv',index=False)

<br>

## 3.3. Join Table Review & Create New Features

In [70]:
review = pd.read_csv('data/review_sentiment.csv')

### 3.3.1. Subset data

In [71]:
# Subset data according to master1
review = review[review.business_id.isin(master1.business_id.unique())]

### 3.3.2. Aggregate data

In [72]:
# Aggregate data to business_id level
review_agg = review.groupby('business_id').aggregate({'stars':np.mean,'useful':np.sum,'funny':np.sum,
                                         'cool':np.sum,'review_sentiment_score':np.mean}
                                       ).rename(columns={'stars':'avg_stars', 'useful':'useful_review_count',
                                                        'funny':'funny_review_count','cool':'cool_review_count'}
                                               ).reset_index()

review_agg.shape

(49369, 6)

In [73]:
review_agg.business_id.nunique()

49369

### 3.3.3. Merge data

In [74]:
# Master file with info from review table
master2 = master1.merge(review_agg.drop(columns='avg_stars'), how='left',on='business_id')

In [75]:
master2.shape

(49369, 47)

In [76]:
# Store temp master file
master2.to_csv('data/master2.csv',index=False)

<br>

## 3.4. Join Table Tip & Create New Features

In [77]:
tip = pd.read_csv('data/tip_sentiment.csv')

### 3.4.1. Subset data

In [78]:
# Subset data according to master1
tip = tip[tip.business_id.isin(master1.business_id.unique())]

In [79]:
tip.isna().sum()

user_id                0
business_id            0
text                   0
compliment_count       0
tip_sentiment_score    0
dtype: int64

### 3.4.2. Aggregate data

In [80]:
tip_agg = tip.groupby('business_id').aggregate({'compliment_count':np.mean,
                                                'tip_sentiment_score':np.mean,
                                                'user_id':np.count_nonzero}
                                       ).rename(columns={'compliment_count':'avg_tip_compliment',
                                                         'user_id':'tip_count'}
                                               ).reset_index()

tip_agg.shape

(44411, 4)

In [81]:
tip.business_id.nunique()

44411

In [82]:
master2.shape

(49369, 47)

In [83]:
tip_agg.head()

,business_id,avg_tip_compliment,tip_sentiment_score,tip_count
0,---kPU91CF4Lq2-WlRu9Lw,0.0,1.000000,4
1,--0iUa4sNDFiZFrAdIWhZQ,0.0,0.500000,6
2,--8IbOsAAxjKRoYsBFL-PA,0.0,0.750000,4
3,--ZVrH2X2QXBFdCilbirsw,0.0,0.750000,8
4,--epgcb7xHGuJ-4PUeSLAw,0.0,0.666667,6


### 3.4.3. Merge data

In [84]:
# Master file with info from tip table
master3 = master2.merge(tip_agg, how='left', on='business_id')

In [85]:
master3.shape

(49369, 50)

In [86]:
s = master3.isna().sum()
s[s>0]

avg_tip_compliment     4958
tip_sentiment_score    4958
tip_count              4958
dtype: int64

In [87]:
master3['has_tip'] = np.where(master3.avg_tip_compliment.isna(),0,1)

In [88]:
# Convert missing value to zero
master3.avg_tip_compliment = np.where(master3.avg_tip_compliment.isna(),0,master3.avg_tip_compliment)
master3.tip_sentiment_score = np.where(master3.tip_sentiment_score.isna(),0,master3.tip_sentiment_score)
master3.tip_count = np.where(master3.tip_count.isna(),0,master3.tip_count)

In [89]:
master3.isna().sum().sum()

0

In [90]:
# Store temp master file
master3.to_csv('data/master3.csv',index=False)

<br>

## 3.5. Further Subset Data to Focus on 4 Cuisines & Create Features

### 3.5.1. Extract cuisine info & check for MECE

In [91]:
master3['Chinese'] = np.where(master3.categories.str.lower().str.contains('chinese')
                 & ~(master3.categories.str.lower().str.contains('fusion')), 1, 0)

master3['Japanese'] = np.where(master3.categories.str.lower().str.contains('japanese')
                 & ~(master3.categories.str.lower().str.contains('fusion')), 1, 0)

master3['Italian'] = np.where(master3.categories.str.lower().str.contains('italian')
                 & ~(master3.categories.str.lower().str.contains('fusion')), 1, 0)

master3['Mexican'] = np.where(master3.categories.str.lower().str.contains('mexican')
                 & ~(master3.categories.str.lower().str.contains('fusion')), 1, 0)

master3['MECE_check'] = master3['Chinese'] + master3['Japanese'] + master3['Italian'] + master3['Mexican']

In [92]:
master4 = master3[master3.MECE_check ==1].copy()
print(master4.shape)

(12005, 56)


In [93]:
master4['cuisine'] = np.where(master4.Chinese, 'Chinese',
                              np.where(master4.Japanese, 'Japanese',
                                      np.where(master4.Italian, 'Italian',
                                              'Mexican')))
master4['cuisine'].value_counts()

Mexican     4427
Italian     4261
Chinese     2262
Japanese    1055
Name: cuisine, dtype: int64

In [94]:
master4.groupby('cuisine').stars.mean()

cuisine
Chinese     3.343501
Italian     3.505867
Japanese    3.761137
Mexican     3.511181
Name: stars, dtype: float64

### 3.5.2. Quicly check differences in cuisines

In [95]:
master4['plant_based'] = np.where(master4.categories.str.lower().str.contains('vegan|vegetarian'), 
                                 1, 0)

master4.groupby('plant_based').stars.mean()

plant_based
0    3.493134
1    3.871981
Name: stars, dtype: float64

In [96]:
master4.groupby('cuisine').plant_based.mean()

cuisine
Chinese     0.011936
Italian     0.013377
Japanese    0.013270
Mexican     0.024622
Name: plant_based, dtype: float64

In [97]:
master4['seafood'] = np.where(master4.categories.str.lower().str.contains('seafood'), 
                                 1, 0)

master4.groupby('seafood').stars.mean()

seafood
0    3.486141
1    3.702937
Name: stars, dtype: float64

In [98]:
master4.groupby('cuisine').seafood.mean()

cuisine
Chinese     0.076923
Italian     0.068059
Japanese    0.054028
Mexican     0.051502
Name: seafood, dtype: float64

In [99]:
master4['gluten_free'] = np.where(master4.categories.str.lower().str.contains('gluten-free'), 
                                 1, 0)

master4.groupby('gluten_free').stars.mean()

gluten_free
0    3.496125
1    3.815789
Name: stars, dtype: float64

In [100]:
master4.groupby('cuisine').gluten_free.mean()

cuisine
Chinese     0.001768
Italian     0.024173
Japanese    0.004739
Mexican     0.004744
Name: gluten_free, dtype: float64

In [101]:
master4['fast_food'] = np.where(master4.categories.str.lower().str.contains('fast food'), 
                                 1, 0)

master4.groupby('fast_food').stars.mean()

fast_food
0    3.614092
1    2.500000
Name: stars, dtype: float64

In [102]:
master4.groupby('cuisine').fast_food.mean()

cuisine
Chinese     0.062334
Italian     0.056794
Japanese    0.010427
Mexican     0.189519
Name: fast_food, dtype: float64

In [103]:
master4['breakfast'] = np.where(master4.categories.str.lower().str.contains('breakfast'), 
                                 1, 0)

master4.groupby('breakfast').stars.mean()

breakfast
0    3.531828
1    3.086505
Name: stars, dtype: float64

In [104]:
master4.groupby('cuisine').breakfast.mean()

cuisine
Chinese     0.005747
Italian     0.042713
Japanese    0.004739
Mexican     0.150666
Name: breakfast, dtype: float64

In [105]:
master4['nightlife'] = np.where(master4.categories.str.lower().str.contains('nightlife'), 
                                 1, 0)

master4.groupby('nightlife').stars.mean()

nightlife
0    3.476459
1    3.680718
Name: stars, dtype: float64

In [106]:
master4.groupby('cuisine').nightlife.mean()

cuisine
Chinese     0.014589
Italian     0.152312
Japanese    0.076777
Mexican     0.135758
Name: nightlife, dtype: float64

### 3.5.3. Create a new feature for restaurant category count

In [107]:
master4['ctgy_count'] = master4['categories'].str.split(',').apply(lambda x: len(x))

master4.groupby('ctgy_count').stars.mean()

ctgy_count
2     3.488417
3     3.456930
4     3.552860
5     3.433939
6     3.280273
7     3.664773
8     3.857713
9     3.780423
10    3.796154
11    3.871429
12    3.938776
13    4.108696
14    4.142857
15    4.000000
16    4.062500
17    4.166667
18    4.250000
19    4.000000
20    4.000000
22    4.000000
36    4.500000
Name: stars, dtype: float64

In [108]:
master4.groupby('cuisine').ctgy_count.mean()

cuisine
Chinese     2.938992
Italian     4.701948
Japanese    4.017062
Mexican     4.232663
Name: ctgy_count, dtype: float64

<br>

## 3.6. Convert Some Nominal Categorical Variables to Binary Variables

In [109]:
master4.groupby('NoiseLevel').stars.mean()

NoiseLevel
average    3.533355
loud       3.189024
quiet      3.505183
unknown    3.489050
Name: stars, dtype: float64

In [110]:
master4.NoiseLevel.value_counts()

average    6026
unknown    3379
quiet      2026
loud        574
Name: NoiseLevel, dtype: int64

In [111]:
master4['noise_loud'] = np.where(master4.NoiseLevel=='loud', 1, 0)

In [112]:
master4.groupby('RestaurantsPriceRange').stars.mean()

RestaurantsPriceRange
1     3.273851
2     3.590934
3     3.707921
4     3.454545
99    3.785455
Name: stars, dtype: float64

In [113]:
master4.RestaurantsPriceRange.value_counts()

2     5460
1     4570
99    1650
3      303
4       22
Name: RestaurantsPriceRange, dtype: int64

In [114]:
master4['expensive'] = np.where(master4.RestaurantsPriceRange.isin([3,4]), 1, 0)

<br>

## 3.6. Store data for subsequent use

In [115]:
# Store temp master file
master4.to_csv('data/master4.csv',index=False)